In [3]:
import cv2
import numpy as np

# Set up the camera capture
cap = cv2.VideoCapture('sample_video.mp4')

# Create a Lucas-Kanade optical flow object
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Initialize variables
old_frame = None
old_gray = None
p0 = None

#defining Saving parameters
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('Final_sparse.avi', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if old_frame is None:
        # If this is the first frame, initialize the variables
        old_frame = frame
        old_gray = gray
        p0 = cv2.goodFeaturesToTrack(old_gray, maxCorners=100, qualityLevel=0.3, minDistance=7)
        mask = np.zeros_like(old_frame)

    else:
        # Calculate optical flow using Lucas-Kanade method
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None, **lk_params)

        # Select only the good points
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Draw the movement vectors
        if len(good_new) > 0 and len(good_old) > 0:
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
                frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 0, 255), -1)

            # Display the resulting image
            img = cv2.add(frame, mask)
            cv2.imshow('frame', img)

            # Update the previous frame and previous points
            old_gray = gray.copy()
            p0 = good_new.reshape(-1, 1, 2)

            #writing the file
            out.write(img)

    # Wait for key press and break if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close the window
cap.release()
cv2.destroyAllWindows()


In [4]:
cap.release()